In [1]:
import cv2
import torch
import random
import numpy as np
from PIL import Image
from torch import optim
from collections import deque
import torch.nn.functional as F
import matplotlib.pyplot as plt

from torch.utils.data import DataLoader
from dataset import Conv_Att_MCI_Dataset
from models import BaseModel, VGG16GradCAM, ConvAttnModel

In [2]:
SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [3]:
# Dataset
img_type = 'all'

# Model
backbone_freezing = True
## Conv-Att
h_dim_attn = 128
n_heads = 1
h_dim_fc = 512
n_layers = 1

# training
batch_size = 32#64
n_epochs = 100

# optimizer
lr = 1e-5
beta_1 = 0.9
beta_2 = 0.99
eps = 1e-7

## Early stopping
es_size = 4

In [4]:
dataset = Conv_Att_MCI_Dataset(img_type)
dataset_trn, dataset_val, dataset_test = dataset.split_trn_val_test()
dataloader_trn = DataLoader(dataset_trn, batch_size=batch_size, shuffle=True)
dataloader_val = DataLoader(dataset_val, batch_size=batch_size, shuffle=True)
dataloader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=False)

In [5]:
# model = BaseModel(img_type, backbone_freezing).to(device)
# model = VGG16GradCAM(img_type, backbone_freezing).to(device)
model = ConvAttnModel(img_type, h_dim_attn, n_heads, h_dim_fc, n_layers, backbone_freezing).to(device)

/root/miniconda3/envs/cdt/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/root/miniconda3/envs/cdt/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/root/miniconda3/envs/cdt/lib/python3.8/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.num_heads is odd
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [6]:
optimizer = optim.Adam(model.parameters(), lr, [beta_1, beta_2], eps)

In [7]:
results = {
    'losses':{'trn':[], 'val':[], 'test':[]},
    'corrects':{'trn':[], 'val':[], 'test':[]},
    'accs':{'trn':[], 'val':[], 'test':[]},
}
es_queue, es_flag = deque(maxlen=es_size), False
for epoch in range(n_epochs):
    _losses, _corrects, n_tot = [],[],0
    model.train()
    for x, y, info in dataloader_trn:
        for i in range(len(x)):
            x[i] = x[i].to(device)
        y = y.to(device)

        y_pred = model(x)
        y_prob = y_pred.softmax(-1)[:,1]
        loss = F.binary_cross_entropy(y_prob, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        _correct = ((y_prob.cpu() >= 0.5) == info['labels']).sum()

        _losses.append(loss.item())
        _corrects.append(_correct)
        n_tot += len(x[-1])
    
    accs = sum(_corrects) / n_tot
    results['losses']['trn'].append(np.mean(_losses))
    results['corrects']['trn'].append(sum(_corrects))
    results['accs']['trn'].append(accs)

    _losses, _corrects, n_tot = [],[],0
    model.eval()
    for x, y, info in dataloader_val:
        for i in range(len(x)):
            x[i] = x[i].to(device)
        y = y.to(device)

        y_pred = model(x)
        y_prob = y_pred.softmax(-1)[:,1]
        loss = F.binary_cross_entropy(y_prob, y)

        _correct = ((y_prob.cpu() >= 0.5) == info['labels']).sum()

        _losses.append(loss.item())
        _corrects.append(_correct)
        n_tot += len(x[-1])
    
    accs = sum(_corrects) / n_tot
    results['losses']['val'].append(np.mean(_losses))
    results['corrects']['val'].append(sum(_corrects))
    results['accs']['val'].append(accs)

    _losses, _corrects, n_tot = [],[],0
    model.eval()
    for x, y, info in dataloader_test:
        for i in range(len(x)):
            x[i] = x[i].to(device)
        y = y.to(device)

        y_pred = model(x)
        y_prob = y_pred.softmax(-1)[:,1]
        loss = F.binary_cross_entropy(y_prob, y)

        _correct = ((y_prob.cpu() >= 0.5) == info['labels']).sum()

        _losses.append(loss.item())
        _corrects.append(_correct)
        n_tot += len(x[-1])
    
    accs = sum(_corrects) / n_tot
    results['losses']['test'].append(np.mean(_losses))
    results['corrects']['test'].append(sum(_corrects))
    results['accs']['test'].append(accs)  

    es_queue.append(results['losses']['val'][-1])
    if len(es_queue) >= es_size:
        if (np.diff(es_queue) >= 0).all() and (np.diff(results['losses']['trn'][-es_size:]) < 0).all():
            es_flag = True

    print("| Epoch %d/%d |"%(epoch+1, n_epochs), end=' Early stopping!\n' if es_flag else '\n')
    print("| Train      | Loss %6.2f | Acc. %6.2f |"%(results['losses']['trn'][-1], results['accs']['trn'][-1]))
    print("| Validation | Loss %6.2f | Acc. %6.2f |"%(results['losses']['val'][-1], results['accs']['val'][-1]))
    print("| Test       | Loss %6.2f | Acc. %6.2f |"%(results['losses']['test'][-1], results['accs']['test'][-1]))

    if es_flag:
        break

| Epoch 1/100 |
| Train      | Loss   0.70 | Acc.   0.64 |
| Validation | Loss   0.67 | Acc.   0.71 |
| Test       | Loss   0.68 | Acc.   0.69 |
| Epoch 2/100 |
| Train      | Loss   0.65 | Acc.   0.70 |
| Validation | Loss   0.64 | Acc.   0.71 |
| Test       | Loss   0.64 | Acc.   0.77 |
| Epoch 3/100 |
| Train      | Loss   0.61 | Acc.   0.73 |
| Validation | Loss   0.60 | Acc.   0.70 |
| Test       | Loss   0.65 | Acc.   0.65 |
| Epoch 4/100 |
| Train      | Loss   0.59 | Acc.   0.76 |
| Validation | Loss   0.60 | Acc.   0.75 |
| Test       | Loss   0.62 | Acc.   0.75 |
| Epoch 5/100 |
| Train      | Loss   0.57 | Acc.   0.76 |
| Validation | Loss   0.59 | Acc.   0.70 |
| Test       | Loss   0.60 | Acc.   0.73 |
| Epoch 6/100 |
| Train      | Loss   0.57 | Acc.   0.77 |
| Validation | Loss   0.59 | Acc.   0.70 |
| Test       | Loss   0.58 | Acc.   0.75 |
| Epoch 7/100 |
| Train      | Loss   0.56 | Acc.   0.76 |
| Validation | Loss   0.57 | Acc.   0.73 |
| Test       | Loss   0.57 |

In [ ]:
fig = plt.figure(figsize=[10,5])
ax = fig.add_subplot(1,2,1)
ax.plot(results['losses']['trn'], label='Trn.')
ax.plot(results['losses']['val'], label='Val.')
ax.plot(results['losses']['test'], label='Test')
ax.legend()
ax.set_title('Loss')

ax = fig.add_subplot(1,2,2)
ax.plot(results['accs']['trn'], label='Trn.')
ax.plot(results['accs']['val'], label='Val.')
ax.plot(results['accs']['test'], label='Test')
ax.legend()
ax.set_title('Accuracy')

In [ ]:
def patch_attention(m):
    forward_orig = m.forward

    def wrap(*args, **kwargs):
        kwargs["need_weights"] = True
        kwargs["average_attn_weights"] = False

        return forward_orig(*args, **kwargs)

    m.forward = wrap
attn_layers = []
for i in range(len(img_type)):
    attn_layers.append(model.attns[i].layers[-1].self_attn)
    patch_attention(attn_layers[-1])

In [ ]:
vgg16_features, attn_inputs, attn_outputs = {_type:[] for _type in img_type}, {_type:[] for _type in img_type}, {_type:[] for _type in img_type}
hooks = []
for i in range(len(img_type)):
  hooks.append(model.vgg16_models[i][-1].register_forward_hook(
      lambda self, input, output: vgg16_features.append(output)
    ))
  hooks.append(attn_layers[i].register_forward_hook(
      lambda self, input, output: attn_inputs.append(input[0])
    ))
  hooks.append(attn_layers[i].register_forward_hook(
      lambda self, input, output: attn_outputs.append(output)
    ))

# # propagate through the model
# outputs = model(x)

# for hook in hooks:
#     hook.remove()

In [ ]:
idx = 130
img = dataset_test.dataset['images'][idx:idx+1].to(device)
score = dataset_test.dataset['scores'][idx:idx+1]
label = dataset_test.dataset['labels'][idx:idx+1]
print(img.shape, score, label)

y_pred = model(img)
# Grad-CAM
# class_idx = y_pred.argmax(-1).item()
# heatmap = model.generate_cam(img, class_idx).cpu()
# print(heatmap.shape)

# Self-attention
for hook in hooks:
    hook.remove()
B,C,H,W = vgg16_features[0].shape
attn_weights = attn_outputs[0][1]
heatmap = attn_weights[:,0,0,1:].reshape(B,H,W).detach().cpu()
heatmap = torch.clamp(heatmap, min=0)
heatmap /= heatmap.max()

heatmap = np.uint8(255 * heatmap)
heatmap = np.uint8(Image.fromarray(heatmap[0]).resize((img.shape[2], img.shape[3])))
heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
print(heatmap.shape)

# alpha = 1.0
# superimposed_img = heatmap * alpha #+ img[0].permute(1,2,0).detach().cpu().numpy()
# superimposed_img = np.clip(superimposed_img, 0, 255).astype(np.uint8)

plt.imshow(img[0].detach().cpu().permute(1,2,0))
plt.imshow(heatmap, alpha=0.5)
plt.colorbar()
plt.axis('off')
plt.title('Pred: %d vs. GT: %d'%(y_pred.argmax(-1).item(), label.item()))
plt.show()